In [373]:
import pandas as pd
import os
from datetime import datetime, timedelta

In [346]:
# Get test data
url_tests = '../static/data_sources/us_states_covid19_daily.csv'
df_tests = pd.read_csv(url_tests).fillna(0)

# Get COVID data
confirmed_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv'
df_confirmed = pd.read_csv(confirmed_url)
cured_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv'
df_cured = pd.read_csv(cured_url)
deceased_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv'
df_deceased = pd.read_csv(deceased_url)

# Get census data
census_url = '../../acs2017_census_tract_data.csv'
df_census = pd.read_csv(census_url)

# Get political data
url_politics = 'https://en.wikipedia.org/wiki/Political_party_strength_in_U.S._states'
df_tables = pd.read_html(url_politics)

# Get States and Emergency Dates data
url_states = '../static/data_sources/states.csv'
df_states = pd.read_csv(url_states,header=None)

In [347]:
df_tests['positive'] = df_tests['positive'].astype(int)
df_tests['negative'] = df_tests['negative'].astype(int)
df_tests['pending'] = df_tests['pending'].astype(int)
df_tests['hospitalized'] = df_tests['hospitalized'].astype(int)
df_tests['death'] = df_tests['death'].astype(int)
df_tests['date'] = pd.to_datetime(df_tests['date'],  format='%Y%m%d')
df_tests = df_tests.drop('dateChecked', axis=1)
df_tests.head()

,date,state,positive,negative,pending,hospitalized,death,total
0,2020-03-22,AK,22,946,0,0,0,968
1,2020-03-22,AL,138,1464,0,0,0,1602
2,2020-03-22,AR,165,711,119,13,0,995
3,2020-03-22,AS,0,0,0,0,0,0
4,2020-03-22,AZ,152,282,87,0,2,521


In [348]:
df_politics = df_tables[3].iloc[:,0:2]
df_politics.rename(columns={'State':'state_name', '2016 presidentialelection':'pol_party'}, inplace=True)
df_states = df_states.drop(0, axis=1)
df_states.rename(columns={1:'state_name', 2 :'state'}, inplace=True)
# df_states.head()

In [349]:
df_confirmed = df_confirmed[df_confirmed['Country/Region']=='US']
df_cured = df_cured[df_cured['Country/Region']=='US']
df_deceased = df_deceased[df_deceased['Country/Region']=='US']

In [350]:
df_confirmed = df_confirmed.melt(id_vars=["Province/State", "Country/Region","Lat","Long"], 
            var_name="Date", 
            value_name="Value")
df_cured = df_cured.melt(id_vars=["Province/State", "Country/Region","Lat","Long"], 
            var_name="Date", 
            value_name="Value")
df_deceased = df_deceased.melt(id_vars=["Province/State", "Country/Region","Lat","Long"], 
            var_name="Date", 
            value_name="Value")
# df_confirmed.head()

In [351]:
df_confirmed.rename(columns={'Province/State':'state_name','Lat':'lat','Long':'long', 'Date':'date','Value':'confirmed'}, inplace=True)
df_confirmed.drop('Country/Region', axis=1)
df_cured.rename(columns={'Province/State':'state_name','Lat':'lat','Long':'long', 'Date':'date','Value':'cured'}, inplace=True)
df_cured.drop('Country/Region', axis=1)
df_deceased.rename(columns={'Province/State':'state_name','Lat':'lat','Long':'long', 'Date':'date','Value':'deceased'}, inplace=True)
df_deceased.drop('Country/Region', axis=1)

# df_confirmed_states.to_csv(output_path)
df_confirmed.head()

,state_name,Country/Region,lat,long,date,confirmed
0,Washington,US,47,-121,1/22/20,0
1,New York,US,42,-75,1/22/20,0
2,California,US,36,-120,1/22/20,0
3,Massachusetts,US,42,-72,1/22/20,0
4,Diamond Princess,US,35,140,1/22/20,0


In [352]:
# output_path = os.path.join("../static/output", "df_confirmed.csv")
df_confirmed_states = pd.merge(df_states, df_confirmed, on='state_name')
df_cured_states = pd.merge(df_states, df_cured, on='state_name')
df_deceased_states = pd.merge(df_states, df_deceased, on='state_name')
df_confirmed_states.head()

,state_name,state,3,Country/Region,lat,long,date,confirmed
0,Alabama,AL,2020-03-13,US,32,-87,1/22/20,0
1,Alabama,AL,2020-03-13,US,32,-87,1/23/20,0
2,Alabama,AL,2020-03-13,US,32,-87,1/24/20,0
3,Alabama,AL,2020-03-13,US,32,-87,1/25/20,0
4,Alabama,AL,2020-03-13,US,32,-87,1/26/20,0


In [353]:
df_confirmed_states.drop(df_confirmed_states.columns[[3]], axis=1, inplace=True)
# df_confirmed_states['date'] = pd.to_datetime(df_confirmed_states['date']).datetime.strptime(last_date, "%m/%d/%Y")
df_cured_states.drop(df_cured_states.columns[[3]], axis=1, inplace=True)
df_deceased_states.drop(df_deceased_states.columns[[3]], axis=1, inplace=True)
df_confirmed_states.head()

,state_name,state,3,lat,long,date,confirmed
0,Alabama,AL,2020-03-13,32,-87,1/22/20,0
1,Alabama,AL,2020-03-13,32,-87,1/23/20,0
2,Alabama,AL,2020-03-13,32,-87,1/24/20,0
3,Alabama,AL,2020-03-13,32,-87,1/25/20,0
4,Alabama,AL,2020-03-13,32,-87,1/26/20,0


In [354]:
df_confirmed_states['date'] = pd.to_datetime(df_confirmed_states['date'], format='%m/%d/%y')
df_cured_states['date'] = pd.to_datetime(df_cured_states['date'], format='%m/%d/%y')
df_deceased_states['date'] = pd.to_datetime(df_deceased_states['date'], format='%m/%d/%y')
df_confirmed_states.head()

,state_name,state,3,lat,long,date,confirmed
0,Alabama,AL,2020-03-13,32,-87,2020-01-22,0
1,Alabama,AL,2020-03-13,32,-87,2020-01-23,0
2,Alabama,AL,2020-03-13,32,-87,2020-01-24,0
3,Alabama,AL,2020-03-13,32,-87,2020-01-25,0
4,Alabama,AL,2020-03-13,32,-87,2020-01-26,0


In [355]:
df_confirmed_states = df_confirmed_states.set_index(['date', 'state','state_name'])
df_cured_states = df_cured_states.set_index(['date', 'state','state_name'])
df_deceased_states = df_deceased_states.set_index(['date', 'state','state_name'])

df_merged = pd.merge(df_confirmed_states, df_cured_states, how='left', on=['date', 'state', 'state_name','lat','long',3])
df_merged = pd.merge(df_merged, df_deceased_states,how='left', on=['date', 'state','state_name','lat','long',3])
df_merged.rename(columns={3:'emergency_date'}, inplace=True)

In [356]:
df_merged.head()
df_merged = df_merged.reset_index()


In [357]:
df_merged_shuffle = df_merged[['state_name','date','state','lat','long','confirmed','cured','deceased','emergency_date']]

In [358]:
output_path = os.path.join("../static/output", "all_merged.csv")
df_merged.to_csv(output_path)

In [359]:
df_total_1 = pd.merge(df_merged, df_politics, how='left', on=['state_name'])
df_total_1.head()

,date,state,state_name,emergency_date,lat,long,confirmed,cured,deceased,pol_party
0,2020-01-22,AL,Alabama,2020-03-13,32,-87,0,0,0,Republican
1,2020-01-23,AL,Alabama,2020-03-13,32,-87,0,0,0,Republican
2,2020-01-24,AL,Alabama,2020-03-13,32,-87,0,0,0,Republican
3,2020-01-25,AL,Alabama,2020-03-13,32,-87,0,0,0,Republican
4,2020-01-26,AL,Alabama,2020-03-13,32,-87,0,0,0,Republican


In [360]:
df_total_2 = pd.merge(df_total_1, df_tests, how='left', on=['date','state'])
df_total_2.fillna(0)

,date,state,state_name,emergency_date,lat,long,confirmed,cured,deceased,pol_party,positive,negative,pending,hospitalized,death,total
0,2020-01-22,AL,Alabama,2020-03-13,32,-87,0,0,0,Republican,0,0,0,0,0,0
1,2020-01-23,AL,Alabama,2020-03-13,32,-87,0,0,0,Republican,0,0,0,0,0,0
2,2020-01-24,AL,Alabama,2020-03-13,32,-87,0,0,0,Republican,0,0,0,0,0,0
3,2020-01-25,AL,Alabama,2020-03-13,32,-87,0,0,0,Republican,0,0,0,0,0,0
4,2020-01-26,AL,Alabama,2020-03-13,32,-87,0,0,0,Republican,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3106,2020-03-18,WY,Wyoming,2020-03-13,43,-107,15,0,0,Republican,15,178,0,0,0,193
3107,2020-03-19,WY,Wyoming,2020-03-13,43,-107,18,0,0,Republican,18,271,0,0,0,289
3108,2020-03-20,WY,Wyoming,2020-03-13,43,-107,19,0,0,Republican,19,331,0,0,0,350
3109,2020-03-21,WY,Wyoming,2020-03-13,43,-107,23,0,0,Republican,23,438,0,0,0,461


In [361]:
df_total_2['positive'] = pd.to_numeric(df_total_2['positive'].fillna(0)).astype(int)
df_total_2['negative'] = pd.to_numeric(df_total_2['negative'].fillna(0)).astype(int)
df_total_2['pending'] = pd.to_numeric(df_total_2['pending'].fillna(0)).astype(int)
df_total_2['hospitalized'] = pd.to_numeric(df_total_2['hospitalized'].fillna(0)).astype(int)
df_total_2['death'] = pd.to_numeric(df_total_2['death'].fillna(0)).astype(int)
df_total_2['total'] = pd.to_numeric(df_total_2['total'].fillna(0)).astype(int)
df_total_2.head()

,date,state,state_name,emergency_date,lat,long,confirmed,cured,deceased,pol_party,positive,negative,pending,hospitalized,death,total
0,2020-01-22,AL,Alabama,2020-03-13,32,-87,0,0,0,Republican,0,0,0,0,0,0
1,2020-01-23,AL,Alabama,2020-03-13,32,-87,0,0,0,Republican,0,0,0,0,0,0
2,2020-01-24,AL,Alabama,2020-03-13,32,-87,0,0,0,Republican,0,0,0,0,0,0
3,2020-01-25,AL,Alabama,2020-03-13,32,-87,0,0,0,Republican,0,0,0,0,0,0
4,2020-01-26,AL,Alabama,2020-03-13,32,-87,0,0,0,Republican,0,0,0,0,0,0


In [362]:
df_census.head()

,TractId,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,...,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
0,1001020100,Alabama,Autauga County,1845,899,946,2,86,5,0,...,0,0,2,24,881,74,21,4,0,5
1,1001020200,Alabama,Autauga County,2172,1167,1005,1,42,54,0,...,0,0,0,22,852,76,15,9,0,3
2,1001020300,Alabama,Autauga County,3385,1533,1852,8,61,26,1,...,1,1,2,23,1482,73,21,5,1,5
3,1001020400,Alabama,Autauga County,4267,2001,2266,10,80,7,0,...,2,3,2,26,1849,76,20,4,0,6
4,1001020500,Alabama,Autauga County,9965,5054,4911,1,78,16,0,...,1,0,1,21,4787,71,24,4,0,2


In [365]:
df_census = df_census.rename(columns=({'State':'state_name'}))
df_census['hispanic'] = round(df_census['Hispanic']*df_census['TotalPop']/100)
df_census['white'] = round(df_census['White']*df_census['TotalPop']/100)
df_census['black'] = round(df_census['Black']*df_census['TotalPop']/100)
df_census['native'] = round(df_census['Native']*df_census['TotalPop']/100)
df_census['asian'] = round(df_census['Asian']*df_census['TotalPop']/100)
df_census['pacific'] = round(df_census['Pacific']*df_census['TotalPop']/100)
df_census['income'] = round(df_census['IncomePerCap']*df_census['TotalPop'])
df_census['poverty'] = round(df_census['Poverty']*df_census['TotalPop']/100)

pd.options.display.float_format = '{:,.0f}'.format

In [366]:
df_census.head()

,TractId,state_name,County,TotalPop,Men,Women,Hispanic,White,Black,Native,...,FamilyWork,Unemployment,hispanic,white,black,native,asian,pacific,income,poverty
0,1001020100,Alabama,Autauga County,1845,899,946,2,86,5,0,...,0,5,44,"1,592",96,0,22,0,"60,918,210",197
1,1001020200,Alabama,Autauga County,2172,1167,1005,1,42,54,0,...,0,3,24,904,"1,184",0,22,0,"41,259,312",487
2,1001020300,Alabama,Autauga County,3385,1533,1852,8,61,26,1,...,1,5,271,"2,078",897,20,24,14,"71,883,860",498
3,1001020400,Alabama,Autauga County,4267,2001,2266,10,80,7,0,...,0,6,410,"3,426",303,21,9,0,"119,766,156",98
4,1001020500,Alabama,Autauga County,9965,5054,4911,1,78,16,0,...,0,2,90,"7,723","1,634",0,309,0,"367,758,325","1,216"


In [367]:
df_census_gb = df_census.groupby('state_name')
df_census_aggr = df_census_gb.agg({
    'TotalPop': 'sum',
    'Men': 'sum',
    'Women': 'sum',
    'hispanic':'sum',
    'white':'sum',
    'black':'sum',
    'native':'sum',
    'asian':'sum',
    'pacific':'sum',
    'Employed':'sum',
    'Income': 'sum',
    'Poverty': 'sum'
})

In [368]:
df_census_aggr

,TotalPop,Men,Women,hispanic,white,black,native,asian,pacific,Employed,Income,Poverty
state_name,,,,,,,,,,,,
Alabama,4850771,2350806,2499965,"198,306","3,198,112","1,281,049","22,595","62,036","1,456",2055509,"53,931,462","23,975"
Alaska,738565,386319,352246,"50,193","454,495","22,572","101,687","44,745","8,795",354045,"12,176,465","1,844"
Arizona,6809946,3385055,3424891,"2,103,476","3,786,436","276,092","266,620","207,272","12,009",2953891,"87,359,329","26,559"
Arkansas,2977944,1461651,1516293,"213,263","2,174,304","457,156","16,962","41,887","6,981",1276536,"30,219,518","13,387"
California,38982847,19366579,19616268,"15,105,839","14,777,803","2,161,560","137,749","5,427,935","138,214",17993915,"581,498,746","123,077"
Colorado,5436519,2731315,2705204,"1,157,253","3,731,912","209,843","28,673","161,796","7,460",2760076,"85,741,898","15,141"
Connecticut,3594478,1754046,1840432,"551,956","2,446,086","350,892","5,176","154,963",664,1805086,"64,812,649","9,292"
Delaware,943732,456876,486856,"84,766","594,889","201,469","2,876","36,085",204,441513,"13,762,455","2,830"
District of Columbia,672391,319046,353345,"71,809","241,908","315,165","1,149","25,080",246,357701,"14,679,725","3,323"


In [369]:
df_total_3 = pd.merge(df_total_2, df_census_aggr, how='left', on=['state_name'])

In [376]:
# df_check = df_total_3[df_total_3['state']=='MO']
df_check = df_total_3[df_total_3['date']=='2020-03-22']

df_check.head()

,date,state,state_name,emergency_date,lat,long,confirmed,cured,deceased,pol_party,...,Women,hispanic,white,black,native,asian,pacific,Employed,Income,Poverty
60,2020-03-22,AL,Alabama,2020-03-13,32,-87,138,0,0,Republican,...,2499965,"198,306","3,198,112","1,281,049","22,595","62,036","1,456",2055509,"53,931,462","23,975"
121,2020-03-22,AK,Alaska,2020-03-11,61,-152,21,0,0,Republican,...,352246,"50,193","454,495","22,572","101,687","44,745","8,795",354045,"12,176,465","1,844"
182,2020-03-22,AZ,Arizona,2020-03-11,34,-111,152,0,2,Republican,...,3424891,"2,103,476","3,786,436","276,092","266,620","207,272","12,009",2953891,"87,359,329","26,559"
243,2020-03-22,AR,Arkansas,2020-03-11,35,-92,165,0,0,Republican,...,1516293,"213,263","2,174,304","457,156","16,962","41,887","6,981",1276536,"30,219,518","13,387"
304,2020-03-22,CA,California,2020-03-04,36,-120,1642,0,30,Democratic,...,19616268,"15,105,839","14,777,803","2,161,560","137,749","5,427,935","138,214",17993915,"581,498,746","123,077"
